In [1]:
import sqlite3
from datetime import datetime

Define the data

In [2]:
# Define multiple rows of data
data = [
    # "https://example.com/job_offer1",
    # "https://example.com/job_offer2",
    "https://example.com/job_offer3",
    "https://example.com/job_offer4",
    "https://example.com/job_offer5",
]

Check if the table exists, if not - create it

In [3]:
try:
    # Connect to the SQLite database
    job_offers_db = sqlite3.connect("job_offers_db.db")
    cursor = job_offers_db.cursor()

    # Query sqlite_master to check if the "offers" table exists
    cursor.execute(
        """
        SELECT name
        FROM sqlite_master 
        WHERE type='table' AND name='offers';
        """
    )

    # Fetch the result
    result = cursor.fetchone()

    if not result:
        cursor.execute(
            """
            CREATE TABLE "offers" (
                "offer_id" INTEGER NOT NULL UNIQUE,
                "link" TEXT NOT NULL,
                "date_created" TEXT NOT NULL,
                "date_updated" TEXT NOT NULL,
                "active" INTEGER NOT NULL,
                PRIMARY KEY("offer_id" AUTOINCREMENT)
            )
            """
        )

        # Commit the transaction
        job_offers_db.commit()

except Exception as e:
    print(f"Error while connecting to a database: \n{type(e).__name__}: {e}")

Insert to or update existing rows in a table

In [4]:
for link in data:
    # Check if data row already exists in the database by the link
    cursor.execute(
        """
        SELECT * 
        FROM offers
        WHERE link = ?
        """,
        (link,),
    )

    if cursor.fetchone():  # If the row exists update it, else create a new one
        try:
            cursor.execute(
                """
                UPDATE offers
                SET 
                    date_updated = ?
                """,
                (datetime.today().strftime("%d-%m-%y"),),
            )

            # Commit the transaction
            job_offers_db.commit()

        except Exception as e:
            print(f"Error udpating a row: \n{type(e).__name__}: {e}")
    else:
        try:
            cursor.execute(
                """
                INSERT INTO offers (link, date_created, date_updated, active)
                    VALUES (?, ?, ?, ?)
                """,
                (
                    link,
                    datetime.today().strftime("%d-%m-%y"),
                    datetime.today().strftime("%d-%m-%y"),
                    1,
                ),
            )

            # Commit the transaction
            job_offers_db.commit()

        except Exception as e:
            print(f"Error creating a row: \n{type(e).__name__}: {e}")

Set the state of the offer to false if date_updated is not a current date

In [5]:
try:
    cursor.execute(
    """
    UPDATE offers
    SET
        active = CASE WHEN date_updated = ? THEN 1 ELSE 0 END
    """,
        (datetime.today().strftime("%d-%m-%y"),),
    )
except Exception as e:
    print(f"Error updating a state of an offer: \n{type(e).__name__}: {e}")

In [6]:
# Close the cursor and connection
cursor.close()
job_offers_db.close()